# Programmieraufgabe IV     ( 6 + 10 + 4 = 20 Punkte) 
## Abgabe in der Woche 17. bis 21. Dezember

In [1]:
import numpy as np 
import matplotlib.pyplot as plt
import numpy.linalg as la

## Teilaufgabe a) 

Schreiben Sie eine Funktion, die eine gegebene Matrix $A$ (numpy-array) mittels Householder-Transformationen in eine obere Hessenbergmatrix überführt. 


In [2]:
# Das ist die Kopie von altem Code

def householder(arg, w): 
    
    ## Diese Funktion soll eine Householder-Transformation $I - 2ww^T$ auf das Argument arg (Matrix oder Vektor) anwenden.
    
    l = len(w)
    
    H_ = np.identity(l)
    for i in range(l):
        for j in range(l):
            H_[i][j] -= 2*w[i]*w[j]
    
    (num_rows, num_cols) = np.shape(arg)
    
    H = np.identity(num_rows)
    H[num_rows-l:, num_rows-l:] = H_
    
    result = H@arg
    
    return (result, H)

In [3]:
# Diese Funktion funktioniert noch nicht.
# Die Dimensionen stimmen nicht ueberein.

def intohessenberg(A):
    
    m = np.shape(A)[0] - 2
    
    for k in range(m):
        
        v = np.copy(A[...,k][k+1:]) # A[k+1:m][k]
        v[0] += np.sign(v[0]) * np.linalg.norm(v)
        v = v / np.linalg.norm(v)
        
        ## Benutzte entweder die alte Funktion von oben:
        A,_ = householder(A,v)
        
        ## Debug output
        # print("k =", k)
        # print("v =", v)
        # print("np.transpose(v) =", np.transpose(v))
        # print("A[...,k:][k+1:] =", A[...,k:][k+1:])
        # print("np.transpose(v) @ A[...,k:][k+1:] =", np.transpose(v) @ A[...,k:][k+1:])
        # print("v @ (np.transpose(v) @ A[...,k:][k+1:]) =", v @ (np.transpose(v) @ A[...,k:][k+1:]))
        
        ## Oder schreibe die Funktion neu:
        # A[...,k:][k+1:] -= 2 * (v @ (np.transpose(v) @ A[...,k:][k+1:])) # A[k+1:m][k:m]
        # A[...,k+1:][k:] -= 2 * ((A[...,k+1:][k:] @ v) @ np.transpose(v)) # A[1:m][k+1:m]
        
        # print("")
        
    return A

In [4]:
A = [[1,2,3],[4,5,6],[7,8,9],[10,11,12],[13,14,15],[16,17,18],[19,20,21]]
A = np.array(A).astype(float)

print(intohessenberg(A))

IndexError: index 3 is out of bounds for axis 1 with size 3

## Teilaufgabe b)

Implementieren Sie das in der Vorlesung behandelte QR-Verfahren zur Eigenwertbestimmung einer oberen Hessenbergmatrix $A$ (numpy-array). Input der Funktion soll die Matrix $A$, eine maximale Anzahl von Iterationen $\text{itmax}$ und eine Toleranz $\text{tol}$ sein, die in ein (sinnvolles) Abbruchkriterium eingeht.

Ausgegeben werden soll ein array mit den Näherungen für die Eigenwerte, geordnet nach Größe. Außerdem soll eine Liste der Eigenwertnäherungen aller Iterationen ausgegeben werden. 

Beachten Sie, dass hier die __in der Vorlesung besprochene praktische Umsetzung des QR-Verfahrens zur Eigenwertbestimmung__ implementiert werden soll, d.h. ein bloßes Ausrechnen der QR-Zerlegung $A=QR$ und anschließende Zuweisung $A \leftarrow RQ$, explizites Aufstellen und Ausmultiplizieren von Givens-Rotations-Matrizen etc. soll hier _nicht_ verwendet werden!

In [5]:
def qr_eigenvalues(A, itmax, tol):
    
    ## to be completed ##
    
    return eigvals, eigvalslist

Testen Sie Ihre Implementierung, indem Sie Ihre Ergebnisse für eine zufällige symmetrische $10 \times 10$-Matrix und die übliche Matrix $A = \text{tridiag}(-1,4,-1) \in \mathbb{R}^{5 \times 5}$ mit der in numpy vorimplementierten Eigenwertberechnung (wählen Sie eine geeignete Routine) vergleichen. 

In [6]:
scal = 10 # Skaliere die Zufallszahlen -> [-scal,scal)

# np.random.random_sample((m,n)) gives B \in [0,1)^(m x n)
# https://docs.scipy.org/doc/numpy/reference/generated/numpy.random.random.html
B = 2*scal*(np.random.random_sample((10,10)) - .5)
B = (B + B.T)/2

A = [[4,-1,0,0,0],
     [-1,4,-1,0,0],
     [0,-1,4,-1,0],
     [0,0,-1,4,-1],
     [0,0,0,-1,4]]
A = np.array(A).astype(float)

print(B)
print(A)

[[ 2.40604667  2.85777295 -0.32696867  3.70245298  7.26366679 -4.89243996
  -3.66081887 -1.22788368 -3.9676795  -6.28515257]
 [ 2.85777295 -5.13926362 -2.53497169  0.82540933 -1.67964657  0.2501766
   0.6344695  -4.29001542 -4.55870887  2.30514985]
 [-0.32696867 -2.53497169 -1.36198636 -3.74157333  1.00061366 -3.29488944
  -2.62216441  0.68295915  5.06851717 -5.13681696]
 [ 3.70245298  0.82540933 -3.74157333  8.9408121   4.16514476 -8.44189906
  -0.50649962  0.32410388 -0.92603304 -5.54896404]
 [ 7.26366679 -1.67964657  1.00061366  4.16514476 -3.59862349  8.00527062
  -4.19233843  2.92300632  1.0824948   3.70758741]
 [-4.89243996  0.2501766  -3.29488944 -8.44189906  8.00527062 -6.06149765
   2.49583757  9.49002733 -4.68864592 -9.66311224]
 [-3.66081887  0.6344695  -2.62216441 -0.50649962 -4.19233843  2.49583757
   8.54627225  0.13770504  8.6116958   3.83000082]
 [-1.22788368 -4.29001542  0.68295915  0.32410388  2.92300632  9.49002733
   0.13770504 -1.4176214  -3.96758916 -4.17726326]
 

In [7]:
itmax = 100
tol = .001

myeigvalsA, _ = qr_eigenvalues(A, itmax, tol)
myeigvalsB, _ = qr_eigenvalues(B, itmax, tol)

NameError: name 'eigvals' is not defined

In [8]:
# The column v[:,i] is the eigenvector corresponding to the eigenvalue w[i]
# https://docs.scipy.org/doc/numpy/reference/generated/numpy.linalg.eig.html

w1, v1 = la.eig(A)
w2, v2 = la.eig(B)
eigvalsA = w1
eigvalsB = w2

print("A:")
for i in range(w1.shape[0]):
    print("w_" + str(i), "=", w1[i])
    print("v_" + str(i), "=", v1[:,i])
print("B:")
for i in range(w2.shape[0]):
    print("w_" + str(i), "=", w2[i])
    print("v_" + str(i), "=", v2[:,i])

A:
w_0 = 2.26794919243
v_0 = [-0.28867513 -0.5        -0.57735027 -0.5        -0.28867513]
w_1 = 3.0
v_1 = [  5.00000000e-01   5.00000000e-01   3.42143760e-16  -5.00000000e-01
  -5.00000000e-01]
w_2 = 4.0
v_2 = [  5.77350269e-01   4.90435668e-17  -5.77350269e-01   4.10676918e-16
   5.77350269e-01]
w_3 = 5.73205080757
v_3 = [-0.28867513  0.5        -0.57735027  0.5        -0.28867513]
w_4 = 5.0
v_4 = [ -5.00000000e-01   5.00000000e-01   5.56426531e-16  -5.00000000e-01
   5.00000000e-01]
B:
w_0 = -26.4331303022
v_0 = [-0.2805701   0.09377316 -0.16385062 -0.25847598  0.45408771 -0.65671322
  0.13310825  0.12079476 -0.16785572 -0.34767446]
w_1 = 23.0855583111
v_1 = [ 0.39130436 -0.00793919  0.04745726  0.31927073  0.21411704  0.15783951
 -0.47384586  0.21471156 -0.29563659 -0.5602147 ]
w_2 = -13.8719891173
v_2 = [-0.09930041 -0.43812672  0.26561504  0.07549429  0.17125721  0.12437069
  0.27796621 -0.50765527 -0.5846568  -0.01827297]
w_3 = -7.80923374435
v_3 = [ 0.27240243 -0.68916905 -0.58

## Teilaufgabe c) 

Erstellen Sie einen Plot, aus dem für jeden Eigenwert der Matrix die Konvergenzgeschwindigkeit der jeweiligen Eigenwertnäherungen des QR-Verfahrens hervorgeht. Als die "richtigen" Eigenwerte können Sie beispielsweise die mit einer numpy-Routine berechneten Eigenwerte oder die letzte Iterierte Ihres Verfahrens verwenden. 

Test-Matrix soll $B = B(a,b,c,d)$ wie unten angegeben sein. 

Variieren Sie die Parameter $a,b,c,d$ und interpretieren Sie Ihre Beobachtungen.

In [9]:
a = 1
b = 2 
c = 3
d = 4

T = 0.5*np.array([[1,1,1,1],[1,1,-1,-1],[-1,1,-1,1],[-1,1,1,-1]])
B = np.diag([a, b, c, d])
B = T.T @ (B @ T) # np.matmul(T.T, np.matmul(B, T))

solution, _ = la.eig(B)
_, eigvalslist = qr_eigenvalues(B, itmax, tol)
print(solution)

eigvalplot = [np.linalg.norm(eigval - solution) for eigval in eigvallist]


plt.plot(eigvalplot, label="eigval B")

plt.legend()
plt.show()
plt.close()

NameError: name 'eigvals' is not defined